In [ ]:
# Health DGA Train Recipe

This notebook demonstrates the use of AI Model Factory recipes to train DGA Model.

### Issues and To-Do Items

* The loop of keeping tell the exeuction status, it might be just get the old messgae without fresh?

* How to check the status after gain the job ID? -

    https://tenant1.predict.masinst1.ibmmam.com/ibm/modelfactory/service/log/$job_id

* To ensure the connection to the deployed site if you are outside of IBM network (or even in the network), us VPN at the CISCO client with this link - http://sasvpn.us.ibm.com/TUNNELALL



# Import Libraries & Functions

In [ ]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [ ]:
# Print functions
def print_job_status(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    status_data = get_response.json()
    
    # Print the job status
    if "status" in status_data:
        print("The status of job {} is {}.".format(job_id, status_data['status']))
    else:
        print(status_data)
    
def print_all_status(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    status_data = get_response.json()
    
    # Print the job status
    print(status_data)


In [ ]:
# More print functions using display 
from IPython.display import display, HTML
import requests

def print_job_details(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    summary_data = get_response.json()
    
    # Display the job status
    display(HTML(print_keys_and_values(summary_data)))
    return summary_data

    
def print_keys_and_values(json_data):
    # Start the HTML code
    html_code = "<div style='font-family: Arial; font-size: 1.2em;'>"
    
    # Add the job details to the HTML code
    html_code += f"<p>job_id: {json_data['job_id']}</p>"
    if "status" in json_data:
        html_code += f"<p>status: {json_data['status']}</p>"
    html_code += "<br>"
    if "detailed_summary" in json_data:
        for summary in json_data['detailed_summary']:
            html_code += f"<p>run_id: {summary['run_id']}</p>"
            html_code += f"<p>experiment_id: {summary['experiment_id']}</p>"
            if "status" in summary:
                html_code += f"<p>status: {summary['status']}</p>"
            html_code += f"<p>artifact_uri: {summary['artifact_uri']}</p>"
            html_code += f"<p>artifact_name: {summary.get('tags.artifact_name', 'No artifact_name found')}</p>"
            html_code += "<br>"
    
    # Close the HTML code
    html_code += "</div>"
    
    return html_code


# Health DGA (Dissolved Gas Analysis) Training

## Defining Paths

In [ ]:
# Define the file paths
# endpoint_url = "https://tenant1.predict.masinst1.ibmmam.com/ibm/modelfactory/service"
endpoint_url = "http://localhost:8000/ibm/modelfactory/service"
data_file_path =  "../data/training.csv"


In [ ]:
feature_columns = "DGAR-H2,DGAR-O2,DGAR-N2,DGAR-CH4,DGAR-CO,DGAR-CO2,DGAR-C2H4,DGAR-C2H6,DGAR-C2H2,DBDS,POWER_FACT,INTER_V,DI_RIG,H2O"
target_columns = "Health index"

## POST Response

In [ ]:
import requests

files = {
    "data_file": ("data/training.csv", open(data_file_path, 'rb')),   
}

data = {
    "feature_columns":feature_columns,
    "target_columns":target_columns,
}

headers = {
  'accept': 'application/json'
}

#url = endpoint_url + "/recipe/electrical_transformer/health/dga/train"
url = endpoint_url + "/recipe/electrical-transformer/health/dga"
post_response = requests.request("POST", url, headers=headers, data=data, files=files)

In [ ]:
post_r_json = post_response.json()
anomaly_service_jobId = None

if 'jobId' in post_r_json:
    anomaly_service_jobId = post_r_json['jobId']
    print ('submitted successfully job : ', post_r_json['jobId'])
else:
    print (post_r_json)


## GET Response  - Keep Refresh it for a while until seeing the STATUS changed to DONE

The status of the job may be running, flagged by INITALIZING or EXECUTING

After a while the model recipe training is complete, and the STATUS changes to DONE

In [ ]:
log_url = endpoint_url + "/log/"
job_id = post_r_json['job_id']

In [ ]:
print(log_url + job_id)

In [ ]:
json_data = print_job_details(job_id,endpoint_url)

In [ ]:
json_data

In [ ]:
s3_folder_name = 'best_estimator'

In [ ]:
import yaml
if 'status' in json_data:
    if json_data['status'] == 'DONE':
        model_uri = json_data['summary']['model_uri'] 
        for summary in json_data["detailed_summary"]:
            if summary['tags.step']!="":
                s3_uri = summary["artifact_uri"] + "/" + s3_folder_name
                print(summary['tags.step'])
                print(s3_uri)
        output_data = {
            "model_uri" : model_uri,
            "train_job_id" : job_id,
            "s3_uri": s3_uri,
        }
        with open("../config/model_info.yml","w") as file:
            yaml.dump(output_data, file)
        output_data

In [ ]:
output_data